# Knowledge-coupled biological entities for lookups and typing

Before ingesting any data, let us review a last concept: knowledge-coupled entities. 

If ingested data isn't curated against public and in-house knowledge-derived standards, data integration becomes the pain it often is.

LaminDB (and the underlying Bionty) offers a few ways to navigate knowledge from lookups, curation, managing knowledge-coupled SQL tables to writing code using the derived types.

Let us curate some data to go illustrate the process.

In [ ]:
import lamindb as ln
import lamindb.schema as lns
import lamindb.knowledge as lnk

ln.nb.header()

## Lookup ontology ids

Module {class}`lamindb.knowledge` offers lookups via tab completion.

For instance, you can retrieve the cell type ontology id of "gamma delta T cell" like so:

In [ ]:
ct_lookup = lnk.CellType().lookup

In [ ]:
ct_lookup.gamma_delta_T_cell

```{note}


See the [Bionty](https://lamin.ai/docs/bionty) documentation for gene name aliasing and other types of lookups.

```

## Create knowledge-derived records

You can also directly create a record for the `CellType` table:

In [ ]:
lns.bionty.CellType(ontology_id=ct_lookup.gamma_delta_T_cell)

## Curate metadata by linking it against knowledge 

Let us link all the biological samples in a cross-tissue scRNA-seq dataset:

In [ ]:
adata = ln.dev.datasets.anndata_human_immune_cells()

meta = adata.obs.drop_duplicates(subset=adata.obs.columns)
meta.shape

In [ ]:
meta.head()

Let's first add all the cell types: `.curate` allows you to check the passed ids are present in the knowledge table.

It returns a new `DataFrame` indexed with the curated ids and a boolean `__curated__` column.

In [ ]:
celltype_curate = lnk.CellType().curate(meta, column="cell_type_ontology_term_id")

Here we saw all terms can be linked. 🎉

## Update the content of the knowledge-managed tables in your SQL database

Assume there are cell types in this new dataset that are tracked in the ontology (`lnk.CellType`), but are not yet tracked in the DB table (`lns.bionty.CellType`).

Let us fix that!

We can go ahead and create records of the `CellType` table:

In [ ]:
celltype_records = [
    lns.bionty.CellType(ontology_id=i) for i in celltype_curate.index.unique()
]

In [ ]:
celltype_records[:3]

We can do the same for tissues:

In [ ]:
tissue_curate = lnk.Tissue().curate(meta, column="tissue_ontology_term_id")

In [ ]:
tissue_records = [
    lns.bionty.Tissue(ontology_id=i) for i in tissue_curate.index.unique()
]

In [ ]:
tissue_records[:3]

Finally, let's add them to the database:

In [ ]:
ln.add(celltype_records + tissue_records)  # add all records in one transaction

Check they are in the database:

In [ ]:
ln.select(lns.bionty.Tissue, name="blood").one()

Now, some of the foundational knowledge is in place.